# DNS Spoofing Detection and Resolution Tool

This notebook implements an enhanced DNS Spoofing Detection and Resolution Tool using Python. It provides capabilities to detect potential DNS spoofing attacks, verify suspicious responses, and trace back to the actual DNS information.

In [4]:
# Import required libraries
import scapy.all as scapy
from scapy.layers.dns import DNS, DNSQR, DNSRR
from scapy.layers.inet import IP, UDP
import socket
import concurrent.futures
import dns.resolver
import whois
from ipwhois import IPWhois

## DNSSpoofDetector Class

This class implements the core functionality for DNS spoofing detection and resolution.

In [5]:
class DNSSpoofDetector:
    def __init__(self, interface):
        self.interface = interface
        self.cache = {}
        self.suspicious_responses = []
        self.stop_detecting = False
        self.resolver = dns.resolver.Resolver()
        self.resolver.nameservers = ['8.8.8.8', '8.8.4.4']  # Using Google's DNS servers

    def dns_detect(self, packet):
        if packet.haslayer(DNSRR):
            qname = packet[DNSQR].qname.decode()
            rdata = packet[DNSRR].rdata

            if qname in self.cache:
                if self.cache[qname] != rdata:
                    print(f"[!] Potential DNS spoofing detected for {qname}")
                    print(f"    Cached IP: {self.cache[qname]}")
                    print(f"    Received IP: {rdata}")
                    self.suspicious_responses.append((qname, self.cache[qname], rdata))
            else:
                self.cache[qname] = rdata

    def verify_dns(self, domain):
        try:
            real_ip = self.resolver.resolve(domain, 'A')[0].to_text()
            cached_ip = self.cache.get(domain)
            if cached_ip and real_ip != cached_ip:
                print(f"[!] Confirmed DNS spoofing for {domain}")
                print(f"    Real IP: {real_ip}")
                print(f"    Spoofed IP: {cached_ip}")
                self.trace_actual_dns(domain, real_ip, cached_ip)
            return real_ip
        except dns.resolver.NXDOMAIN:
            print(f"[!] Domain {domain} does not exist.")
            return None
        except Exception as e:
            print(f"[!] Error resolving {domain}: {e}")
            return None

    def trace_actual_dns(self, domain, real_ip, spoofed_ip):
        print(f"[*] Tracing actual DNS for {domain}")
        
        # Get WHOIS information for the domain
        try:
            domain_info = whois.whois(domain)
            print(f"    Domain Registrar: {domain_info.registrar}")
            print(f"    Name Servers: {', '.join(domain_info.name_servers)}")
        except Exception as e:
            print(f"    Unable to retrieve WHOIS information for domain: {e}")

        # Get information about the real IP
        try:
            ip_info = IPWhois(real_ip).lookup_rdap()
            print(f"    Real IP ASN: {ip_info.get('asn')}")
            print(f"    Real IP ASN Description: {ip_info.get('asn_description')}")
        except Exception as e:
            print(f"    Unable to retrieve information for real IP: {e}")

        # Get information about the spoofed IP
        try:
            spoofed_ip_info = IPWhois(spoofed_ip).lookup_rdap()
            print(f"    Spoofed IP ASN: {spoofed_ip_info.get('asn')}")
            print(f"    Spoofed IP ASN Description: {spoofed_ip_info.get('asn_description')}")
        except Exception as e:
            print(f"    Unable to retrieve information for spoofed IP: {e}")

    def start_detecting(self):
        try:
            print(f"[*] Starting DNS spoofing detection on {self.interface}...")
            scapy.sniff(iface=self.interface, filter="udp port 53", prn=self.dns_detect, stop_filter=self.check_stop)
        except PermissionError:
            print("Error: Permission denied. Try running with administrative privileges.")
        except KeyboardInterrupt:
            print("\n[*] Stopping DNS spoofing detection...")
        except Exception as e:
            print(f"An error occurred: {e}")

    def check_stop(self, packet):
        return self.stop_detecting

    def stop_detection(self):
        self.stop_detecting = True

    def verify_suspicious_responses(self):
        print("[*] Verifying suspicious responses and tracing actual DNS...")
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(self.verify_dns, domain) for domain, _, _ in self.suspicious_responses]
            concurrent.futures.wait(futures)

## Usage Example

Here's an example of how to use the DNSSpoofDetector class. Note that this should be run with administrative privileges to capture network traffic.

In [3]:
interface = 'Wi-Fi'  # Replace with your network interface
detector = DNSSpoofDetector(interface)

try:
    detector.start_detecting()
except KeyboardInterrupt:
    print("\n[*] Stopping DNS spoofing detection...")
    detector.stop_detection()
finally:
    detector.verify_suspicious_responses()
    print("[*] DNS spoofing detection stopped. Exiting...")

[*] Starting DNS spoofing detection on Wi-Fi...
[!] Potential DNS spoofing detected for a1887.dscq.akamai.net.
    Cached IP: 49.44.118.100
    Received IP: 2405:200:1602::312c:7664
[!] Potential DNS spoofing detected for a1887.dscq.akamai.net.
    Cached IP: 49.44.118.100
    Received IP: 2405:200:1602::312c:7664
[!] Potential DNS spoofing detected for prod.content-signature-chains.prod.webservices.mozgcp.net.
    Cached IP: 34.160.144.191
    Received IP: 2600:1901:0:92a9::
[!] Potential DNS spoofing detected for prod.content-signature-chains.prod.webservices.mozgcp.net.
    Cached IP: 34.160.144.191
    Received IP: 2600:1901:0:92a9::
[!] Potential DNS spoofing detected for www.google.com.
    Cached IP: 142.250.182.36
    Received IP: 2404:6800:4007:804::2004
[!] Potential DNS spoofing detected for pki-goog.l.google.com.
    Cached IP: 142.250.196.163
    Received IP: 2404:6800:4007:82c::2003
[!] Potential DNS spoofing detected for tiles-cdn.prod.ads.prod.webservices.mozgcp.net.
  

## Notes

- Ensure you have the required libraries installed: `scapy`, `dnspython`, `python-whois`, `ipwhois`.
- Replace `'eth0'` with your actual network interface name.
- This tool is for educational purposes and should only be used in environments where you have explicit permission to monitor network traffic.
- Always respect privacy and legal considerations when working with network tools.